## Szöveggenerálás a 'Cunk on Earth' című brit sorozat feliratfájljainak felhasználásával háromféle modellel

#### Készítette: Mészáros Dominik
#### Neptun kód: EZU0EX

***

## Első modell

##### A modell létrehozásához szükségek modulok importálása

In [ ]:
import numpy as np
import tensorflow as tf
import random
import sys
import matplotlib.pyplot as plt

##### A felhasználandó szöveg betöltése

In [ ]:
with open("data/data.txt", "r") as f:
    text = f.read().lower()
print(len(text))
text[:100]

##### Adott hosszúságú szövegrészleteket adott számú eltolással egy listába rak,
##### majd a szövegrészleteket követő karaktereket egy újabb listába helyez
##### Ez azért kell, hogy a modell ne az egész szöveget egyszerre akarja feldolgozni, hanem részenként, de mégis legyen egy szövegkörnyezet amiben egy adott szövegrész beletartozik

In [ ]:
maxlen = 60
step = 3
sequences = []
next_chars = []

for i in range(0, len(text) - maxlen):
    sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
len(sequences)

##### Minden egyedi karakterhez egy azonosítót tarsít

In [ ]:
chars = sorted(list(set(text)))
char_index = dict((char, chars.index(char)) for char in chars)
index_char = dict((chars.index(char), char) for char in chars)

In [ ]:
x = []
y = []

for i, sequence in enumerate(sequences):
    x.append([char_index[char] for char in sequence])
    y.append([char_index[next_chars[i]]])
x = np.array(x)
y = np.array(y)

##### Létrehozzuk a modellt, majd betanítjuk a betöltött adattal

In [ ]:
emb_size = 8
num_chars = len(chars)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_chars, emb_size, input_length=maxlen),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=num_chars, activation='softmax')
]) 

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

model.summary()

In [ ]:
with tf.device("/GPU:0"):
    history = model.fit(x, y, epochs=25, batch_size=32)

##### A training loss és epoch függvényének szemléltetése

In [ ]:
plt.plot(history.history['loss'], label='(training data)',color='blue')
plt.title('Neural Network training loss')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.show()

##### A modell egy véletlenszerűen kiválasztott szövegrészletet folytat a tanultak alapján

In [ ]:
start = random.randint(0, len(sequences))
base_text = sequences[start]
base_text

In [ ]:
next_char = 200
new_text = base_text
sys.stdout.write(new_text)

for _ in range(next_char):
    prediction = np.argmax(model.predict(np.array([[char_index[char] for char in new_text]]), verbose=0), axis=-1)
    new_char = index_char[prediction[0]]
    new_text = new_text[1:] + new_char
    sys.stdout.write(new_char)

## Konklúzió

#### A modell 25 epoch tanítás után 0.4121 pontosságot (1.9330 loss) tudott elérni. A tizedik epoch után sokat lassul a tanulás. Sok a szóismétlés, nem tud új ötletekkel előállni, ugyanazokat a szókapcsolatokat ismétli. 

***

## Második modell - Long Short-Term Memory használatával

##### A modell létrehozásához szükséges modulok importálása

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
import sys

##### A felhasználandó szöveg betöltése, illetve tokenizációja

In [ ]:
with open("data/data.txt", "r") as f:
    text = f.read().lower()

print(len(text))

lines = text.split("\n")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
num_words = len(tokenizer.word_index) + 1

In [ ]:
print(lines[3])
tokenizer.texts_to_sequences([lines[3]])[0]

In [ ]:
input_sequences = []

for line in lines:
    tokens = tokenizer.texts_to_sequences([line])[0]

    for i in range(1, len(tokens)):
        input_sequences.append(tokens[:i+1])

In [ ]:
input_sequences[:10]

##### Az összes szekvenciát feltöltjük 0-val, hogy az összes array 16 elem hosszú legyen

In [ ]:
max_sequence_len = max([len(i) for i in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
np.random.shuffle(input_sequences)
X, y = input_sequences[:,:-1], np.expand_dims(input_sequences[:,-1], axis=1)
X[:5]

In [ ]:
num_words

##### Létrehozzuk a modellt

In [ ]:
emb_size = 256

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_words, emb_size, input_length=max_sequence_len - 1),
    tf.keras.layers.LSTM(120),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(units=num_words, activation='softmax')
]) 

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer="adam", metrics=['accuracy'])

model.summary()

##### Betanítjuk a modellt a feldolgozott adatokkal
##### Az EarlyStopping callback function abban segít, hogyha a modell már nem képes kevesebb veszteséget elérni, akkor a tanulás befejeződik, ezzel időt és erőforrást spórolva

In [ ]:
callback = EarlyStopping(monitor="loss", patience=3)
history = model.fit(X, y, epochs=25, batch_size=128, verbose=1, callbacks=[callback])

##### A training loss és epoch függvényének szemléltetése

In [ ]:
plt.plot(history.history['loss'], label='(training data)',color='blue')
plt.title('Neural Network training loss')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.show()

##### A training accuracy és epoch függvényének szemléltetése

In [ ]:
plt.plot(history.history['accuracy'], label='(training data)',color='blue')
plt.title('Neural Network training accuracy')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.show()

##### A létrehozott modell egy véletlenszerű szövegrészletet fog folytatni

In [ ]:
start = random.randint(0, len(sequences))
base_text = sequences[start]
base_text

In [ ]:
text = base_text
next_words = 100

print(text)
for _ in range(next_words):
    tokens = tokenizer.texts_to_sequences([text])[0]
    tokens = pad_sequences([tokens], maxlen=max_sequence_len-1, padding='pre')   
    predicted = np.argmax(model.predict(tokens, verbose=0), axis=-1)
 
    output_word = ""

    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

    sys.stdout.write(output_word + " ")
    text += " " + output_word

## Konklúzió

#### A modell 25 epoch tanítás után 0.5613 pontosságot (1.9215 loss) tudott elérni. 2-3 szavas értelmes és nyelvtanilag helyes szókapcsolatokat könnyedén tud generálni, viszont ezekből ritkán vagy egyáltalán nem tudott koherens mondatokat létrehozni.

## Harmadik modell rekurrens neurális hálóval

##### A modell elkészítéséhez szükséges modulok importálása

In [ ]:
import tensorflow as tf
import numpy
import random
import sys

##### A tanulásra felhasználandó szöveg beolvasása, majd az első modellnél használt adatelőkészítési módszerrel feldolgozzuk a szöveget

In [ ]:
with open("data/data.txt", "r") as f:
    text = f.read().lower()

print(len(text))

In [ ]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
len(sentences)

In [ ]:
chars = sorted(list(set(text)))
print(chars)

In [ ]:
char_index = dict((char, chars.index(char)) for char in chars)
print(char_index)

In [ ]:
index_char = dict((chars.index(char), char) for char in chars)
print(index_char)

##### A karaktereket vektorizáljuk

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_index[char]] = 1
    y[i, char_index[next_chars[i]]] = 1

print("x", x.shape)
print("y", y.shape)

##### Létrehozzuk a rekurrens neurális hálót, majd betanítjuk az adatokkal

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.GRU(16, input_shape=(maxlen, len(chars))),
    tf.keras.layers.Dense(units=len(chars), activation='softmax')
]) 

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

model.summary()

In [ ]:
with tf.device("/CPU:0"):
    history = model.fit(x, y, batch_size=128, epochs=10)

##### A training loss és epoch függvényének szemléltetése

In [ ]:
plt.plot(history.history['loss'], label='(training data)',color='blue')
plt.title('Neural Network training loss')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.show()

##### A létrehozott modell egy véletlenszerű szövegrészletet fog folytatni

In [ ]:
start = random.randint(0, len(sequences))
base_text = sequences[start]
base_text

In [ ]:
def sample(preds, temperature):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import sys
temperatures = [0.2, 0.5, 1.0, 1.2]
gen_characters = 200

for temp in temperatures:
    print("Temp: ", temp)
    generated_text = base_text
    print(generated_text)
    for i in range(gen_characters):
        sampled = np.zeros((1, maxlen, len(chars)))    
        for t, char in enumerate(generated_text):
            sampled[0, t, char_index[char]] = 1.          
        
        preds = model.predict(sampled, verbose=0)[0]      
        
        next_index = sample(preds, temp)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()

    print()

## Konklúzió

#### A modell 10 epoch tanítás után 1.9405 veszteséget ért el. Különböző temperature-t használva különböző eredményt kapunk. Láthatjuk, hogy a túl kevés, és a túl sok se jó, ezért kell megtalálni egy középértéket, amit a legjobbnak találunk. Itt 4 darab temperature értéket néztünk meg: 
* 0.2 érték mellett értelmes szavakat generál a modellünk, viszont ezek rövidek és sokat ismétlődnek
* 0.5 értéket nézve még mindig értelmes szavakat kapunk, hosszabakat is mint az előző példánál, viszont koherencia csak ritkán figyelhető meg
* 1.0 értéknél is figyelhetünk meg értelmes szavakat, viszont a többi szó nem sorolható az emberi aggyal értelmezhető szavak közé
* 1.2 értéket megadva már csak csekély számmal találhatunk értelmes szavakat, de ezek is csak rövid szavak